In [1]:
import pickle
import pandas as pd

In [2]:
import pickle


archivo_pickle = 'Entrenamiento_cronicas_N.pkl'


with open(archivo_pickle, 'rb') as file:
    df = pickle.load(file)


In [3]:
pd.set_option('display.max_columns', None)
df.shape

(996234, 45)

In [4]:
# Eliminar columnas donde todas las entradas son cero
df = df.loc[:, (df != 0).any(axis=0)]
df['DIAS_COMP'] = df['DIAS_COMP'].astype(float)
df.drop(columns='NUM_IDE', inplace=True)
df.drop(columns='FECHA_NACI', inplace=True)
df.drop(columns='id', inplace=True)
df.drop(columns='MUNI', inplace=True)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Cargar tus datos
# df = pd.read_csv('ruta_a_tu_archivo.csv')

# Eliminar filas con valores nulos
df = df.dropna()

# Binarizar las variables binarias especificadas
binary_columns = [
    'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
    'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
    'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
    'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
    'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
    'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
    'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
    'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
    'CANCER_TERAPIACANCER'
]
df[binary_columns] = df[binary_columns].fillna(0).astype(int)
df = df[df[binary_columns].sum(axis=1) > 0]

# Aplicar One-Hot Encoding a las variables categóricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'V8', 'Grupo_Edad', 'Zona']
encoder = OneHotEncoder()
encoded_categoricals = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_categoricals.toarray(), columns=encoder.get_feature_names_out(categorical_columns), index=df.index)

# Concatenar los datos codificados con el resto del dataframe
df = pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1)

# Dividir el conjunto de datos en entrenamiento y prueba
X = df.drop('Valor_Neto', axis=1)
y = df['Valor_Neto']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

# Ahora X_train, X_test, y_train, y_test están listos para ser usados en el modelado.


In [8]:
X_train.head()

,DIAS_COMP,DIAS_ESTAN_Neto,Edad,SIDA_VIH,TUBERCULOSIS,CANCER_OTROSCANCER,CANCER_ORGANOSDIGESTIVOS,CANCER_ORGANOSRESPIRATORIOS,CANCER_MELANOMAYDELAPIEL,CANCER_MAMA,CANCER_OTROSGENITALESFEMENINOS,CANCER_CERVIXINVASIVO,CANCER_GENITALESMASCULINOS,CANCER_TEJIDOLINFATICOYAFINES,CANCER_CERVIXINSITU,AUTOINMUNE,DIABETES,SINDROMESCONVULSIVOS(EPILEPSIA),ENFERMEDADCARDIOVASCULAR_OTRA,ENFERMEDADCARDIOVASCULAR_HIPERTENSION,ENFERMEDADPULMONARDELARGADURACION,ASMA,ARTRITISPIOGENASYREACTIVAS,ARTRITIS,ARTROSIS,RENAL_OTRA,RENAL_RENALDELARGADURACION,TRANSPLANTE,RENAL_INSUFICIENCIARENALOTRA,RENAL_INSUFICIENCIARENALCRONICA,ANOMALIASGENETICASYCONGENITAS,CANCER_TERAPIACANCER,EPS_EPS001,EPS_EPS002,EPS_EPS003,EPS_EPS005,EPS_EPS008,EPS_EPS009,EPS_EPS010,EPS_EPS012,EPS_EPS013,EPS_EPS016,EPS_EPS017,EPS_EPS018,EPS_EPS023,EPS_EPS037,SEXO_F,SEXO_M,DPTO_11,DPTO_13,DPTO_15,DPTO_17,DPTO_18,DPTO_19,DPTO_20,DPTO_23,DPTO_25,DPTO_27,DPTO_41,DPTO_44,DPTO_47,DPTO_5,DPTO_50,DPTO_52,DPTO_54,DPTO_63,DPTO_66,DPTO_68,DPTO_70,DPTO_73,DPTO_76,DPTO_8,DPTO_81,DPTO_85,DPTO_86,DPTO_88,DPTO_91,DPTO_94,DPTO_95,DPTO_97,DPTO_99,V8_A,V8_B,V8_C,Grupo_Edad_1,Grupo_Edad_10,Grupo_Edad_11,Grupo_Edad_12,Grupo_Edad_2,Grupo_Edad_3,Grupo_Edad_4,Grupo_Edad_5,Grupo_Edad_6,Grupo_Edad_7,Grupo_Edad_8,Grupo_Edad_9,Zona_C,Zona_N,Zona_Z
rownames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3838161,330.0,244.0,30.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1035194,360.0,206.0,35.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1497679,360.0,563.0,66.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1113111,257.0,6.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
189853,90.0,1580.0,49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Inicializar los modelos
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(verbosity=0)
}

# Entrenar y evaluar los modelos
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    results[name] = {
        'train': {
            'R2': r2_score(y_train, y_train_pred),
            'MAE': mean_absolute_error(y_train, y_train_pred),
            'MSE': mean_squared_error(y_train, y_train_pred)
        },
        'test': {
            'R2': r2_score(y_test, y_test_pred),
            'MAE': mean_absolute_error(y_test, y_test_pred),
            'MSE': mean_squared_error(y_test, y_test_pred)
        }
    }

# Imprimir resultados
for model_name, metrics in results.items():
    print(f"Modelo: {model_name}")
    for dataset_type, values in metrics.items():
        print(f"\t{dataset_type} Metrics:")
        for metric_name, metric_value in values.items():
            print(f"\t\t{metric_name}: {metric_value:.4f}")

Modelo: Linear Regression
	train Metrics:
		R2: 0.1410
		MAE: 377441.3542
		MSE: 1290348843672.3057
	test Metrics:
		R2: 0.1194
		MAE: 379759.6614
		MSE: 1500056478771.7434
Modelo: Decision Tree
	train Metrics:
		R2: 0.9998
		MAE: 1305.5475
		MSE: 312467225.5478
	test Metrics:
		R2: -0.5780
		MAE: 434064.3034
		MSE: 2687968435888.4517
Modelo: Random Forest
	train Metrics:
		R2: 0.8811
		MAE: 119865.4805
		MSE: 178587497901.8942
	test Metrics:
		R2: 0.1443
		MAE: 325119.3517
		MSE: 1457710230652.6047
Modelo: XGBoost
	train Metrics:
		R2: 0.7530
		MAE: 271789.8566
		MSE: 371090293777.8057
	test Metrics:
		R2: 0.1033
		MAE: 321450.9364
		MSE: 1527454975749.5811


In [11]:
!pip install optuna

In [12]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'tree_method': 'exact',  # para modelos más pequeños, 'exact' es adecuado
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    model = XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=50)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

best_params = study.best_params
print("Mejores parámetros:", best_params)

# Entrenar el modelo con los mejores parámetros
optimized_model = XGBRegressor(**best_params)
optimized_model.fit(X_train, y_train)

# Evaluación del modelo
y_train_pred = optimized_model.predict(X_train)
y_test_pred = optimized_model.predict(X_test)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"R2 Train: {train_r2:.4f}")
print(f"R2 Test: {test_r2:.4f}")


[I 2024-05-18 10:41:49,038] A new study created in memory with name: no-name-904db49a-c7e2-4565-bd8b-c99a4f64985e
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_48437/1286862207.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_48437/1286862207.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/opt/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stop

Mejores parámetros: {'lambda': 3.745508285393392e-06, 'alpha': 0.005060582683433504, 'colsample_bytree': 0.8703307122347325, 'subsample': 0.949933287494957, 'learning_rate': 0.041256448862423516, 'n_estimators': 923, 'max_depth': 4, 'min_child_weight': 161}
R2 Train: 0.2955
R2 Test: 0.2230
